# Bojective: Forest vs Building Classification using ANN
Steps:
Steps:
1. Importing (or installing) Tenosrflow, Keras and other packages on your system
2. Loading your data from disk
3. Creating your training and testing splits
4. Data Preprocessing
5. Defining your tensorflow ANN model architecture
6. Compiling your tensorflow ANN model
7. Training your model on your training data
8. Evaluating your model on your test data
9. Generate Plots for accuracy and validation loss
10. Saving The train model
11. Making predictions using your trained tensorflow model


## Step 1 : Importing all the packages

In [ ]:
# Import libraries and packages
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
#time.time(); print('Time taken: (:.1f) seconds'.format(time.time() - timel))
import time
#timel
import warnings
from tqdm import tqdm_notebook as tqdm
import itertools


warnings.filterwarnings("ignore")
SEED=42 # set random seed


##Step : 2 Loading ypur data from disk for traning

In [ ]:
#mount google drive to collab notebook
from google.colab import drive
drive.mount("//content/drive")

In [ ]:
# changee working DIR
import os
os.chdir("/content/drive/MyDrive/Deep Learning project/Image classification")

In [ ]:
# # Extract dataset.rar file
#!pip install patool
#import patoolib
#patoolib.extract_archive("dataset.rar")
#patoolib.extract_archive("test_examples.rar")

In [ ]:
# initialize the data and labels
print("[INFO] loading images...")
time1 = time.time()   # to measure time taken
data = []
labels = []
classes =["Forest","Buildings"]

# Grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("dataset")))   # data folder with 2 categorical folders
random.seed(SEED)
random.shuffle(imagePaths)

# Progress bar
with tqdm (total = len(imagePaths)) as pbar:
  for imagePath in imagePaths:
    # Load the image, resize the image to be 32x32 pixels (ignoring aspect ratio),
    # flatten the
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)

    # Extract the class label from the image path
    label = imagePath.split(os.path.sep)[-2]
    label = 1 if label == "Buildings" else 0
    labels.append(label)

    # update the progresbar
    pbar.update(1)

# Scale the raw pixel intensities to the range [0,1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

print("Time taken: {:.1f} seconds".format(time.time() - time1))    # to measure time taken
print("done")

In [ ]:
import pandas as pd
df=pd.DataFrame(data)
df['Label']=labels

In [ ]:
df.head()

In [ ]:
# samples data for first image
print('sample image:{}'.format(data[0]))
print('no of feature/pixels values:{}'.format(len(data[0]))) #32x32x3=3072
print('label:{}'.format(classes[labels[0]]))



In [ ]:
128*128*3


In [ ]:
#partition the data into 80% training and 20% valdiation
(trainX,testX,trainY,testY)=train_test_split(data,labels,test_size=0.2,random_state=SEED)

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
testX.shape

In [ ]:
testY.shape

In [ ]:
trainX

In [ ]:
trainY

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(trainX, trainY)

pred = lr.predict(testX)
print(confusion_matrix(testY, pred))
print(classification_report(testY, pred))
print(accuracy_score(testY, pred))

# STEP 4 : Data preprocessing

In [ ]:
# convert the labels from integers/categories to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

#[0,1] Buldings
#[1,0]Forest

In [ ]:
#testY

In [ ]:
trainY

In [ ]:
sample_image=(trainX[25]*255).astype("int")

In [ ]:
plt.imshow(sample_image.reshape(32,32,3))

In [ ]:
trainY[25] #[0,1] means buildings [1,0] means forest

# Step 5 : define the architecture for ANN model

In [ ]:
# define the 3072-1024-512-1 architecture using Keras
model=Sequential()
#input layer 3072 as there are 32x32x3=3072 pixells in a flattened input image
#first hidden layer has 1024 nodes
model.add(Dense(units=1024,input_shape=(3072,),kernel_initializer="uniform",activation='relu'))

# # droppout for second layer
model .add(Dropout(0.4))

# second hidden layer has 512 nodes
model.add(Dense(units=512,kernel_initializer="uniform",activation='relu'))

#output layer with number of possible class labels
model.add(Dense(units=2,kernel_initializer="uniform",activation='softmax'))



# *Step* 6 : Compiling your tensflow ANN model

In [ ]:
#initialize our inital learning rate and # of epoch to train for
INIT_LR=0.01
EPOCHS=50
#compile the model using SGD as our optimizer and categorical cross_entrophy loss
#(you'll want to use binary_crossentropy for class classifiction)
print('[INFO] compiling network network....')
opt = SGD(learning_rate=INIT_LR)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
model.summary()

# Step 7 : Tranning Youe model

In [ ]:
# train the neural network on training data set
# batch_size (32) controls the size of each group of data to pass through the network

H = model.fit(trainX,trainY,validation_data=(testX, testY),epochs=EPOCHS,batch_size=32)

model.save("ANN_MODEL_{}.h5".format(EPOCHS))

# Step 8: Generate Plots for acc and val

In [ ]:
# plot the training and validation loss
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("ANN: Training & Validation Loss")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Loss", weight="bold")
plt.legend()
plt.show()

In [ ]:
# plot the training and validation accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("ANN: Training and Validation Accuracy")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Accuracy", weight="bold")
plt.legend()
plt.show()

In [ ]:
# Evaluate on test data
loss, accuracy = model.evaluate(testX, testY)
print(f"Accuracy: {accuracy * 100:.2f}%")


# Step 9 : Making Prediction Using your Tensorflow trained Model:

In [ ]:
def display_img(img):
  fig = plt.figure(figsize=(12,10))
  # plt.grid(b=None)
  ax = fig.add_subplot(111)
  ax.imshow(img)

In [ ]:
# load the input image an resize it to the target spatial dimansions:
import imutils
width = 32
height = 32

# grab the image paths and randomly shuffle them
testimagePaths = sorted(list(paths.list_images("test_examples")))
random.seed(SEED)
random.shuffle(testimagePaths)

# Progress bar
with tqdm (total = len(testimagePaths)) as pbar:
  for imagePath in testimagePaths:
    # Load the image, resize the image to be
    image = cv2.imread(imagePath)
    output = image.copy()
    image = cv2.resize(image, (width, height))
    # scale the pixel intensities to the range [0,1]
    image = image.astype("float") / 255.0
    image = image.flatten()
    image = image.reshape((1, image.shape[0]))

    # make a prediction on the image
    preds = model.predict(image)
    i = preds.argmax(axis=1)[0]
    label = classes[i]
    label = "{}: {:.2f}%".format(label, preds[0][i] * 100)
    print("[INFO] {}".format(label))
    output = imutils.resize(output, width=400)
    cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Convert img to rgb format and display in notebook
    img = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    display_img(output)
    pbar.update(1)


In [ ]:
!pip install gradio


# Deployment

In [ ]:
import gradio as gr

In [ ]:
def predict_image(image):

  image = cv2.resize(image, (32,32))

  #scale the pixel values to [0, 1]
  image = image.astype("float") / 255.0

  # for a simple fully-connected network, flatten the image
  image = image.flatten()
  image = image.reshape((1, image.shape[0]))

  # make a predication on the image
  preds =model.predict(image).flatten()
  result = dict()
  result["Forest"] = round(float(list(preds)[0]), 3)
  result["Buildings"] = round(float(list(preds)[1]), 3)
  print(result)

  return result

im = gr.Image()
label = gr.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=im, outputs=label, title="ANN Demo").launch(share=True)